# Constructing The Model

## Imports

In [ ]:
import cv2
import IPython.display as ipd
import librosa # Compatible with python 3.10
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional  as F
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms


from PIL import Image
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from torchvision.transforms import v2

## Constants

#### Directories

In [ ]:
# Assign paths
base_dir = "dataset"

# Use for-norm dataset
train_dir = os.path.join(base_dir, "for-norm", "for-norm", "training")
test_dir = os.path.join(base_dir, "for-norm", "for-norm", "testing")
val_dir = os.path.join(base_dir, "for-norm", "for-norm", "validation")

# Classified directories
train_dir_fake = os.path.join(train_dir, "fake")
train_dir_real = os.path.join(train_dir, "real")
test_dir_fake = os.path.join(test_dir, "fake")
test_dir_real = os.path.join(test_dir, "real")
val_dir_fake = os.path.join(val_dir, "fake")
val_dir_real = os.path.join(val_dir, "real")

In [ ]:
# Check for mistakes
print("Base directory:", base_dir)
print()
print("Train directory:", train_dir)
print("Test directory:", test_dir)
print("Val directory:", val_dir)
print()
print("Train (fake) directory:", train_dir_fake)
print("Train (real) directory:", train_dir_real)
print("Test (fake) directory:", test_dir_fake)
print("Test (real) directory:", test_dir_real)
print("Val (fake) directory:", val_dir_fake)
print("Val (real) directory:", val_dir_real)

#### Other Constants

In [ ]:
# Torch seed (for reproducibility purpose)
SEED = 33

BATCH_SIZE = 16
EPOCH = 20
LEARNING_RATE = 0.0001
NUM_WORKERS = 12
PATIENCE = 5

In [ ]:
torch.cuda.empty_cache() # Free CUDA cache memory
torch.manual_seed(SEED) # Assign Pytorch seed for reproducibility
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Initiate a CUDA device instance, if exists

In [ ]:
# Current model
def get_current_model_path(new_model=False):
    count = 1

    while (True):
        if (os.path.isfile("../../trained_models/audio_convnext_model_" + f"{count}" + ".pth") == False):
            if (new_model):
                return "../../trained_models/audio_convnext_model_" + f"{count}" + ".pth"
            else:
                return "../../trained_models/audio_convnext_model_" + f"{count - 1}" + ".pth"
        else:
            count += 1

## Obtain Data

In [ ]:
# Obtain waveform (.wav) audio files
train_fake_audio_path = [os.path.join(train_dir_fake, file) for file in os.listdir(train_dir_fake) if file.endswith(".wav")]
train_real_audio_path = [os.path.join(train_dir_real, file) for file in os.listdir(train_dir_real) if file.endswith(".wav")]

test_fake_audio_path = [os.path.join(test_dir_fake, file) for file in os.listdir(test_dir_fake) if file.endswith(".wav")]
test_real_audio_path = [os.path.join(test_dir_real, file) for file in os.listdir(test_dir_real) if file.endswith(".wav")]

validation_fake_audio_path = [os.path.join(val_dir_fake, file) for file in os.listdir(val_dir_fake) if file.endswith(".wav")]
validation_real_audio_path = [os.path.join(val_dir_real, file) for file in os.listdir(val_dir_real) if file.endswith(".wav")]

In [ ]:
# Get training labels
train_labels = []

for i in train_fake_audio_path:
    train_labels.append(0)
for i in train_real_audio_path:
    train_labels.append(1)

# Convert list to Pandas dataframe
train_labels_df = pd.DataFrame({'label':train_labels})

# Get testing labels
val_labels = []

for i in validation_fake_audio_path:
    val_labels.append(0)
for i in validation_real_audio_path:
    val_labels.append(1)

# Convert list to Pandas dataframe
val_labels_df = pd.DataFrame({'label':val_labels})

In [ ]:
train_labels_df.head(10)

In [ ]:
val_labels_df.head(10)

In [ ]:
#  Evaluate the training labels
count = 0

for i in range(len(train_labels)):
    if (train_labels[i] == 0):
        count = count+1

print(train_labels)
print("Number of total labels:", len(train_labels))
print("Number of fake audio samples:", count)
print("Number of real audio samples:", len(train_labels) - count)


In [ ]:
#  Evaluate the testing labels
count = 0

for i in range(len(val_labels)):
    if (val_labels[i] == 0):
        count = count+1

print(val_labels)
print("Number of total labels:", len(val_labels))
print("Number of fake audio samples:", count)
print("Number of real audio samples:", len(val_labels) - count)

In [ ]:
# Display a random real training sample audio
random_real_training_audio_file = train_real_audio_path[random.randint(0, 333)]
ipd.Audio(random_real_training_audio_file)

In [ ]:
# Display a random fake training sample audio
random_fake_training_audio_file = train_fake_audio_path[random.randint(0, 333)]
ipd.Audio(random_fake_training_audio_file)


## Data Preprocessing

In [ ]:
# Training
class CustomTrainingAudioDataset(Dataset):
    def __init__(self, real_audio_files, fake_audio_files, target_shape):
        self.real_files = real_audio_files
        self.fake_files = fake_audio_files
        self.target_shape = target_shape
        self.all_files = self.real_files + self.fake_files
        self.labels = [0] * len(self.fake_files) + [1] * len(self.real_files) # 0: fake, 1: real

    def __len__(self):
        return len(self.all_files)

    def __getitem__(self, idx):
        file_path = self.all_files[idx]
        label = self.labels[idx]
        spectrogram = self._create_mel_spectrogram(file_path)
        resized_spectrogram = cv2.resize(spectrogram, self.target_shape[::-1], interpolation=cv2.INTER_LINEAR)
        # Convert to PyTorch tensor and add a channel dimension
        resized_spectrogram = torch.tensor(resized_spectrogram).unsqueeze(0).float()
        label = torch.tensor(label).long()
        return resized_spectrogram, label

    def _create_mel_spectrogram(self, file_path):
        audio_data, sample_rate = librosa.load(file_path)
        mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
        mel_decibel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        return mel_decibel_spectrogram

# Testing
class CustomTestingAudioDataset(Dataset):
    def __init__(self, real_audio_files, fake_audio_files, target_shape):
        self.real_files = real_audio_files
        self.fake_files = fake_audio_files
        self.target_shape = target_shape
        self.all_files = self.real_files + self.fake_files
        self.labels = [0] * len(self.fake_files) + [1] * len(self.real_files) # 0: fake, 1: real

    def __len__(self):
        return len(self.all_files)

    def __getitem__(self, idx):
        file_path = self.all_files[idx]
        label = self.labels[idx]
        spectrogram = self._create_mel_spectrogram(file_path)
        resized_spectrogram = cv2.resize(spectrogram, self.target_shape[::-1], interpolation=cv2.INTER_LINEAR)
        # Convert to PyTorch tensor and add a channel dimension
        resized_spectrogram = torch.tensor(resized_spectrogram).unsqueeze(0).float()
        label = torch.tensor(label).long()
        return resized_spectrogram, label

    def _create_mel_spectrogram(self, file_path):
        audio_data, sample_rate = librosa.load(file_path)
        mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
        mel_decibel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        return mel_decibel_spectrogram

In [ ]:
# Define your target shape
target_shape = (128, 87)

# Create datasets
train_dataset = CustomTrainingAudioDataset(train_real_audio_path, train_fake_audio_path, target_shape)
val_dataset = CustomTrainingAudioDataset(validation_real_audio_path, validation_fake_audio_path, target_shape)
test_dataset = CustomTestingAudioDataset(test_real_audio_path, test_fake_audio_path, target_shape)

In [ ]:
# Display datasets' information
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

In [ ]:
# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

## Designing The Model

In [ ]:
# Load pretrained ConvNeXt Base model
model = models.convnext_base(weights="DEFAULT")

# Freeze the backbone
for param in model.features.parameters():
    param.requires_grad = False

# Modify the first convolutional layer to accept 1 input channel
first_conv_layer = model.features[0][0]
in_channels = first_conv_layer.in_channels
out_channels = first_conv_layer.out_channels
kernel_size = first_conv_layer.kernel_size
stride = first_conv_layer.stride
padding = first_conv_layer.padding
bias = first_conv_layer.bias is not None

new_first_conv_layer = nn.Conv2d(1, out_channels, kernel_size, stride, padding, bias=bias)
model.features[0][0] = new_first_conv_layer

# Replace the classifier head with a custom one
model.classifier = nn.Sequential(
    nn.AvgPool2d(kernel_size=3, padding=1, stride=3),  
    nn.Flatten(),                 
    nn.Linear(1024, 2),              
)


In [ ]:
# Move the model to gpu
model = model.to(device)

In [ ]:
# Define loss function, optimizer, and learning rate scheduler
loss_function = nn.CrossEntropyLoss() # Implement crossentropy loss
optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=LEARNING_RATE) # AdamW
# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4) # Initialize an optimizer
scheduler = StepLR(optimizer, step_size=3, gamma=0.6)

In [ ]:
# Early stopping monitor
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float("inf")
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
def train_loop(dataloader, model, loss_function, optimizer, val_loader=None, early_stopping=None, scheduler=None):
    model.train()
    
    all_predictions = [] # all predictions from the training model after each epoch
    all_labels = [] # all predictions from the training model after each epoch
    train_losses = [] # Array of training losses in each epoch
    lowest_val_loss = 1 # The lowest validation loss after an epoch
    val_losses = [] # Array of validation losses for plotting graph
    val_accuracies = [] # Array of validation accuracies for plotting graph

    current_model_path = get_current_model_path(new_model=True)

    progress_bar_length = 64 # Total number of "#" in the progress bar
    
    for epoch in range(EPOCH):
        progress_accumulated = 0.0 # Fractional progress tracker
        total_hashes = 0 # Total "#" printed so far

        print(f"Training epoch: {epoch + 1}")
        
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, data in enumerate(dataloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # Collecting labels and predictions for classification report
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Update progress bar
            progress_accumulated, total_hashes = progress_bar(
                dataloader, progress_accumulated, total_hashes, progress_bar_length
            )

        # After epoch completes, ensure the progress bar shows 100%
        if total_hashes < progress_bar_length:
            bar = "#" * progress_bar_length
            print(f"Completion (100.00%) : [{bar}]", flush=True)
        else:
            print()  # Move to new line

        epoch_loss = running_loss / len(dataloader)
        accuracy = 100 * correct / total
        
        # Save epoch loss and accuracy
        train_losses.append(epoch_loss)
        
        print(f"\nTraining Loss: {epoch_loss:.4f}, Training Accuracy: {accuracy:.2f}%")
        
        if val_loader:
            val_loss, val_accuracy = validate_model(val_loader, model, loss_function)
            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)

            print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

            save_best_model(lowest_val_loss, val_loss, current_model_path)

            if (val_loss <= lowest_val_loss):
                lowest_val_loss = val_loss

            if early_stopping:
                early_stopping(val_loss)
                if early_stopping.early_stop:
                    print("Early stopping triggered.")
                    break
                
        # Call the scheduler to update the learning rate after each epoch
        if scheduler:
            scheduler.step()

        print("\n", end="\r")

    if val_loader:
        return train_losses, all_predictions, all_labels, val_losses, val_accuracies
    else:
        return train_losses, all_predictions, all_labels

def validate_model(dataloader, model, loss_function):
    model.eval()
    
    correct = 0
    total = 0
    val_loss = 0.0  
    
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    
    return val_loss / len(dataloader), accuracy

def progress_bar(dataloader, accumulated, total_hashes, progress_bar_length):
    total_batches = len(dataloader)
    if total_batches == 0:
        return accumulated, total_hashes
    
    # Calculate progress increment per batch
    progress_per_batch = progress_bar_length / total_batches
    accumulated += progress_per_batch # Add progress for current batch
    
    num_prints = int(accumulated) # Number of "#" to add now
    if num_prints > 0:
        total_hashes += num_prints
        accumulated -= num_prints # Reset accumulated
        
        # Cap at 100% to avoid overshooting
        if total_hashes > progress_bar_length:
            total_hashes = progress_bar_length
        
        # Calculate percentage and print
        percentage = (total_hashes / progress_bar_length) * 100
        bar = "=" * (total_hashes - 1)

        print(f"Completion ({percentage:.2f}%) : [{bar}>]", end="\r", flush=True)
    
    return accumulated, total_hashes

# If the file exists already, change the number behind the file
def save_best_model(lowest_val_loss, current_val_loss, current_model_path):
    if (current_val_loss < lowest_val_loss):
        torch.save(model.state_dict(), current_model_path) # save the trained model

In [ ]:
torch.cuda.empty_cache() # Free CUDA cache memory

if val_dataloader:
    epoch_losses, all_preds, all_labels, val_losses, val_accuracies = train_loop(
        dataloader=train_dataloader,
        model=model,
        loss_function=loss_function,
        optimizer=optimizer,
        val_loader=val_dataloader,
        early_stopping=EarlyStopping(patience=PATIENCE)
    )
else:   
    epoch_losses, all_preds, all_labels = train_loop(
        dataloader=train_dataloader,
        model=model,
        loss_function=loss_function,
        optimizer=optimizer,
        early_stopping=EarlyStopping(patience=PATIENCE)
    )

# Evaluating the Model

In [ ]:
# Loss graph
plt.figure(figsize=(10, 6))
plt.plot(epoch_losses, label="Training Loss") # Training loss
plt.plot(val_losses, label="Validation Loss") # Validation loss
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()    
plt.legend()    
plt.grid(True)

count = 1

while (True):
    if (os.path.isfile("../../trained_models/audio_convnext_model_" + f"{count}" + ".pth") == False):
        plt.savefig("../../src/audio/graphs/audio_convnext_model_" + f"{count - 1}" + "_training_loss.png") # Save the graph
        break
    else:
        count += 1

plt.show()

# Accuracy graph
plt.figure(figsize=(10, 6))
plt.plot(val_accuracies, label="Validation Accuracy") # Validation accuracy
plt.title("Validation accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

count = 1

while (True):
    if (os.path.isfile("../../trained_models/audio_convnext_model_" + f"{count}" + ".pth") == False):
        plt.savefig("../../src/audio/graphs/audio_convnext_model_" + f"{count - 1}" + "_validation_accuracy.png") # Save the graph
        break
    else:
        count += 1

plt.show()

# Calculate the accuracy for the entire training set
train_accuracy = 100 * sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)

print(f"Final Training Accuracy: {train_accuracy:.2f}%")

In [ ]:
current_model_path = get_current_model_path()

In [ ]:
# Load pretrained ConvNeXt Base model
model = models.convnext_base(weights="DEFAULT")

# Freeze the backbone
for param in model.features.parameters():
    param.requires_grad = False

# Modify the first convolutional layer to accept 1 input channel
first_conv_layer = model.features[0][0]
in_channels = first_conv_layer.in_channels
out_channels = first_conv_layer.out_channels
kernel_size = first_conv_layer.kernel_size
stride = first_conv_layer.stride
padding = first_conv_layer.padding
bias = first_conv_layer.bias is not None

new_first_conv_layer = nn.Conv2d(1, out_channels, kernel_size, stride, padding, bias=bias)
model.features[0][0] = new_first_conv_layer

# Replace the classifier head with a custom one
model.classifier = nn.Sequential(
    nn.AvgPool2d(kernel_size=3, padding=1, stride=3),  
    nn.Flatten(),                 
    nn.Linear(1024, 2),              
)

In [ ]:
# Load model
model.load_state_dict(torch.load(current_model_path)) # load the trained model
model = model.to(device)

In [ ]:
model.eval()

# Initialize lists to store predictions and true labels
all_test_preds = []
all_test_labels = []

# Disable gradient calculations for evaluation
with torch.no_grad():
    # Iterate over the test dataloader
    for data in test_dataloader:
        # Get inputs and labels, and move them to the correct device
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(inputs)

        # Get the predicted class (the one with the highest probability)
        _, predicted = torch.max(outputs, 1)

        # Append the predicted classes and true labels to the lists
        # Move predictions and labels to CPU and convert to numpy arrays for scikit-learn
        all_test_preds.extend(predicted.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

In [ ]:
# Define the class names based on the label encoding used in the dataset (0: fake, 1: real)
target_names = ['fake', 'real']

# Generate and print the classification report
print("Classification Report on Test Data:")
report = classification_report(all_test_labels, all_test_preds, target_names=target_names)
print(report)

# Generate and print the confusion matrix
print("\nConfusion Matrix on Test Data:")
cm = confusion_matrix(all_test_labels, all_test_preds)
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()